<a href="https://colab.research.google.com/github/marcelAnd02/MachineLearning/blob/master/Text_gen_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import re
import random
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
raw_data = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/NLP_Project/PoetryFoundationData.csv')
dataset = raw_data.Poem.str.lower().values.tolist()

In [0]:
print(dataset[35])

In [0]:
dataset = list(map(lambda x: re.findall(f"[\w]+|[.,'?!{chr(8217)}]", x), dataset))

In [0]:
num_unique = 2000 #max number of unique words in model's vocab
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = "")
tokenizer.fit_on_texts(dataset)
dataset_tokenized = tokenizer.texts_to_sequences(dataset)

In [0]:
chain_len = 20
X = []
def filter_dataset():
  for ind, ex in enumerate(dataset_tokenized):
    for beg in range(0, len(ex) - chain_len):
      if np.max(ex[beg : beg + chain_len]) <= num_unique:
        X.append(dataset[ind][beg : beg + chain_len])

In [0]:
filter_dataset()

In [0]:
input_lng = 20000
random.shuffle(X)
X = X[0 : input_lng]
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = "")
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

In [0]:
X = np.asarray(X)
X = keras.utils.to_categorical(X)

In [15]:
print(X.shape)
num_unique = X.shape[2]

(20000, 20, 1999)


In [68]:
tokenizer.sequences_to_texts([[127]])[0]

'yet'

## Model

In [0]:
compact_size = 300
encoder = keras.models.Sequential(
    [
     keras.layers.LSTM(128, input_shape = (chain_len, num_unique)),
     keras.layers.Dense(512, activation = "selu"),
     keras.layers.Dense(compact_size, activation = "selu"),
    ]
)
decoder = keras.models.Sequential(
    [
     keras.layers.Dense(512, activation = "selu", input_shape = [compact_size]),
     keras.layers.RepeatVector(chain_len),
     keras.layers.LSTM(128, return_sequences = True),
     keras.layers.TimeDistributed(keras.layers.Dense(num_unique)),
    ]
)
AutoEncoder = keras.models.Sequential([encoder, decoder])
AutoEncoder.compile(optimizer = "adam", loss = "mse")

In [18]:
AutoEncoder.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (None, 300)               1309484   
_________________________________________________________________
sequential_4 (Sequential)    (None, 20, 1999)          740175    
Total params: 2,049,659
Trainable params: 2,049,659
Non-trainable params: 0
_________________________________________________________________


In [19]:
AutoEncoder.fit(X, X, batch_size = 128, epochs = 150)

Epoch 1/150
157/157 [==============================] - 4s 28ms/step - loss: 4.9855e-04
Epoch 2/150
157/157 [==============================] - 4s 28ms/step - loss: 4.9074e-04
Epoch 3/150
157/157 [==============================] - 4s 28ms/step - loss: 4.8704e-04
Epoch 4/150
157/157 [==============================] - 4s 28ms/step - loss: 4.8487e-04
Epoch 5/150
157/157 [==============================] - 4s 28ms/step - loss: 4.8359e-04
Epoch 6/150
157/157 [==============================] - 4s 28ms/step - loss: 4.8253e-04
Epoch 7/150
157/157 [==============================] - 4s 29ms/step - loss: 4.8142e-04
Epoch 8/150
157/157 [==============================] - 4s 28ms/step - loss: 4.8061e-04
Epoch 9/150
157/157 [==============================] - 4s 28ms/step - loss: 4.7994e-04
Epoch 10/150
157/157 [==============================] - 4s 28ms/step - loss: 4.7937e-04
Epoch 11/150
157/157 [==============================] - 4s 28ms/step - loss: 4.7893e-04
Epoch 12/150
157/157 [===================

In [21]:
Xc = X[0].reshape((1, X[0].shape[0], X[0].shape[1]))
print(AutoEncoder.predict(Xc)[0].shape)

(20, 1999)


In [0]:
def build_text(vec):
  pred = []
  for word in vec:
    pos = np.argmax(word)
    if pos == 0:
      continue
    pred.append(tokenizer.sequences_to_texts([[pos]])[0])
  return pred

In [23]:
ind = 5
pred = AutoEncoder.predict(X[ind].reshape((1, X[ind].shape[0], X[ind].shape[1])))[0]
print(build_text(pred))
print(build_text(X[ind]))

['the', 'you', 'you', 'you', 'you', 'you', 'you', 'you', 'you', 'you', '.', 'you', 'a', 'a', 'a', 'a', '’', 's', 'a', 'a']
['the', 'food', 'they', 'keep', 'on', 'giving', 'you', 'makes', 'you', 'sick', '.', 'this', 'hunger', 'is', 'a', 'moment', '’', 's', 'vision', 'that']
